Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using DynamicHMCModels
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff, LinearAlgebra

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_d("..", "scripts", "10")
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);
df[:pulled_left] = convert(Array{Int64}, df[:pulled_left])
df[:prosoc_left] = convert(Array{Int64}, df[:prosoc_left])
df[:condition] = convert(Array{Int64}, df[:condition])
df[:actor] = convert(Array{Int64}, df[:actor])
first(df, 5)

struct m_10_04d_model{TY <: AbstractVector, TX <: AbstractMatrix,
  TA <: AbstractVector}
    "Observations."
    y::TY
    "Covariates"
    X::TX
    "Actors"
    A::TA
    "Number of observations"
    N::Int
    "Number of unique actors"
    N_actors::Int
end

Make the type callable with the parameters *as a single argument*.

In [4]:
function (problem::m_10_04d_model)(θ)
    @unpack y, X, A, N, N_actors = problem   # extract the data
    @unpack β, α = θ  # works on the named tuple too
    ll = 0.0
    ll += sum(logpdf.(Normal(0, 10), β)) # bp & bpC
    ll += sum(logpdf.(Normal(0, 10), α)) # alpha[1:7]
    ll += sum(
      [loglikelihood(Binomial(1, logistic(α[A[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
    ll
end

Instantiate the model with data and inits.

In [5]:
#m_10_04_data = Dict("N" => size(df, 1), "actor" => df[:actor],
#"pulled_left" => df[:pulled_left], "prosoc_left" => df[:prosoc_left],
#"N_actors" => 7, "condition" => df[:condition]);

N = size(df, 1)
N_actors = length(unique(df[:actor]))
X = hcat(ones(Int64, N), df[:prosoc_left] .* df[:condition]);
A = df[:actor]
y = df[:pulled_left]
p = m_10_04d_model(y, X, A, N, N_actors);
θ = (β = [1.0, 0.0], α = [-1.0, 10.0, -1.0, -1.0, -1.0, 0.0, 2.0])
p(θ)

-305.21943396408915

Write a function to return properly dimensioned transformation.

In [6]:
problem_transformation(p::m_10_04d_model) =
    as( (β = as(Array, size(p.X, 2)), α = as(Array, p.N_actors), ) )
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Tune and sample.

In [7]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.013 s/step ...done
MCMC, adapting ϵ (25 steps)
0.011 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0068 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0051 s/step ...done
MCMC, adapting ϵ (200 steps)
0.0028 s/step ...done
MCMC, adapting ϵ (400 steps)
0.0021 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0027 s/step ...done
MCMC (1000 steps)
step 461 (of 1000), 0.0022 s/step
step 907 (of 1000), 0.0022 s/step
0.0022 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [8]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :α),Tuple{Array{Float64,1},Array{Float64,1}}},1}:
 (β = [-1.56031, 0.166343], α = [1.26101, 9.30388, 0.900793, 1.23971, 1.59585, 2.05408, 3.01625])     
 (β = [2.51073, 0.427718], α = [-3.09255, 4.07468, -3.55126, -3.28807, -2.99869, -2.22893, -0.564244])
 (β = [3.18283, 0.490527], α = [-3.55017, 5.35874, -4.01528, -3.87966, -3.57212, -2.73681, -1.16555]) 
 (β = [3.40836, 0.307927], α = [-3.90481, 3.9903, -4.58186, -4.1177, -4.06334, -2.72031, -1.01155])   
 (β = [1.14753, 0.155366], α = [-1.67069, 4.63967, -2.01392, -1.90991, -1.62355, -0.415894, 0.827159])

Extract the parameter posterior means: `β`,

In [9]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 1.5826527798528935
 0.410337188397377 

Extract the parameter posterior means: `β`,

In [10]:
posterior_α = mean(last, posterior)

7-element Array{Float64,1}:
 -2.0307828947719817
 10.298449963678966 
 -2.333207641903682 
 -2.342844814127306 
 -2.0108600036437103
 -1.0969388840935768
  0.4554782972748711

Effective sample sizes (of untransformed draws)

In [11]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×9 Array{Float64,2}:
 643.768  1000.0  647.504  376.221  …  643.916  639.979  650.908  641.31

NUTS-specific statistics

In [12]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.92, min/25%/median/75%/max: 0.01 0.89 0.96 0.99 1.0
  termination: AdjacentDivergent => 0% AdjacentTurn => 24% DoubledTurn => 76%
  depth: 2 => 1% 3 => 43% 4 => 56% 5 => 0%


Result rethinking

In [13]:
rethinking = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
        Mean        SD       Naive SE       MCSE      ESS
a.1 -0.74503184 0.26613979 0.0042080396 0.0060183398 1000
a.2 10.77955494 5.32538998 0.0842018089 0.1269148045 1000
a.3 -1.04982353 0.28535997 0.0045119373 0.0049074219 1000
a.4 -1.04898135 0.28129307 0.0044476339 0.0056325117 1000
a.5 -0.74390933 0.26949936 0.0042611590 0.0052178124 1000
a.6  0.21599365 0.26307574 0.0041595927 0.0045153523 1000
a.7  1.81090866 0.39318577 0.0062168129 0.0071483527 1000
 bp  0.83979926 0.26284676 0.0041559722 0.0059795826 1000
bpC -0.12913322 0.29935741 0.0047332562 0.0049519863 1000
";

Means of draws

In [14]:
[posterior_β, posterior_α]

2-element Array{Array{Float64,1},1}:
 [1.58265, 0.410337]                                                  
 [-2.03078, 10.2984, -2.33321, -2.34284, -2.01086, -1.09694, 0.455478]

End of `10/m10.04d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*